In [2]:
img1, img2=Image.open('1.jpg'),Image.open('2.jpg')
diff=ImageChops.difference(img1,img2)

if diff.getbbox():
    diff.show()
else:
    print('Chưa tìm thấy điểm khác biệt')

In [18]:
#Chuyển định dạng ảnh, có vẻ như khi nén ảnh theo jpg thì nó sẽ chỉnh sửa một số chỗ nên có sự sai sót khi tìm điểm khác nhau
from PIL import Image
im = Image.open('3.jpg')
im.save('3.png')

In [6]:
import cv2 as cv
import numpy as np

# load images
img1 = cv.imread("1.png")
img2 = cv.imread("2.png")

# calculate difference
diff = cv.subtract(img1, img2)  # other order `(img2, img1)` gives worse result

# saves difference
cv.imwrite("difference.png", diff)

# show difference - press any key to close
cv.imshow('diff', diff)
cv.waitKey(0)
cv.destroyWindow('diff')

if not np.any(diff):
    print("The images are the same!")
else:
    print("The images are differant")

# resize images to make them smaller
img1_resized = cv.resize(img1, (0, 0), fx=0.5, fy=0.5)
diff_resized = cv.resize(diff, (0, 0), fx=0.5, fy=0.5)    
img1_resized = img1
diff_resized = diff

# convert to grayscale (without saving and loading again)
diff_resized = cv.cvtColor(diff_resized, cv.COLOR_BGR2GRAY)

# find anything not black in differance
non_zero = cv.findNonZero(diff_resized)
#print(non_zero)

# find top, bottom, left and right edge of the differance
x_min = non_zero[:,0,0].min()
x_max = non_zero[:,0,0].max()
y_min = non_zero[:,0,1].min()
y_max = non_zero[:,0,1].max()
print('x:', x_min, x_max)
print('y:', y_min, y_max)

sizes = [x_max-x_min+1, y_max-y_min+1]
print('width :', sizes[0])
print('height:', sizes[1])

# center 
center_x = (x_min + x_max) // 2
center_y = (y_min + y_max) // 2
center = (center_x, center_y)
print('center:', center)

# radius 
radius = max(sizes) // 2
print('radius:', radius)

color = (0, 0, 255)
thickness = 2

# draw circle around the center of the differance
finished = cv.circle(img1_resized, center, radius, color, thickness)

# saves final image with circle
#cv.imwrite("final.png", finished)

# show final image - press any key to close
cv.imshow('finished', finished)
cv.waitKey(0)
cv.destroyWindow('finished')

The images are differant
x: 0 499
y: 0 666
width : 500
height: 667
center: (249, 333)
radius: 333


In [ ]:
from skimage.metrics import structural_similarity
import cv2
import numpy as np

# Load images
before = cv2.imread('1.jpg')
after = cv2.imread('2.jpg')

# Convert images to grayscale
before_gray = cv2.cvtColor(before, cv2.COLOR_BGR2GRAY)
after_gray = cv2.cvtColor(after, cv2.COLOR_BGR2GRAY)

# Compute SSIM between the two images
(score, diff) = structural_similarity(before_gray, after_gray, full=True)
print("Image Similarity: {:.4f}%".format(score * 100))

# The diff image contains the actual image differences between the two images
# and is represented as a floating point data type in the range [0,1] 
# so we must convert the array to 8-bit unsigned integers in the range
# [0,255] before we can use it with OpenCV
diff = (diff * 255).astype("uint8")
diff_box = cv2.merge([diff, diff, diff])

# Threshold the difference image, followed by finding contours to
# obtain the regions of the two input images that differ
thresh = cv2.threshold(diff, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]
contours = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
contours = contours[0] if len(contours) == 2 else contours[1]

mask = np.zeros(before.shape, dtype='uint8')
filled_after = after.copy()

for c in contours:
    area = cv2.contourArea(c)
    if area > 40:
        x,y,w,h = cv2.boundingRect(c)
        cv2.rectangle(before, (x, y), (x + w, y + h), (36,255,12), 2)
        cv2.rectangle(after, (x, y), (x + w, y + h), (36,255,12), 2)
        cv2.rectangle(diff_box, (x, y), (x + w, y + h), (36,255,12), 2)
        cv2.drawContours(mask, [c], 0, (255,255,255), -1)
        cv2.drawContours(filled_after, [c], 0, (0,255,0), -1)

cv2.imshow('before', before)
cv2.imshow('after', after)
cv2.imshow('diff', diff)
cv2.imshow('diff_box', diff_box)
cv2.imshow('mask', mask)
cv2.imshow('filled after', filled_after)
cv2.waitKey()

C:\Users\admin\anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


Image Similarity: 96.9648%
